# Crashlytics

In [2]:
try:
    import pyspark
except ModuleNotFoundError:
    !pip3 install pyspark
    import pyspark
try:
    import pandas as pd
except ModuleNotFoundError:
    !pip3 install pandas
    import pandas as pd
    import csv
try:
    import matplotlib.pyplot as plt
except ModuleNotFoundError:
    !pip3 install matplotlib
    import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.6/153.6 kB 2.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 9.5 MB/s eta 0:00:00a 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.2/66.2 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.1 MB/s eta 0:00:00


In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, LongType, IntegerType, FloatType
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler, IndexToString, PCA
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [4]:
#May need to install java for this to work
ss=SparkSession.builder.master("local").appName("crashlytics").getOrCreate()

23/11/03 15:25:48 WARN Utils: Your hostname, elliottmac.local resolves to a loopback address: 127.0.0.1; using 104.39.103.165 instead (on interface en0)
23/11/03 15:25:48 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/03 15:25:49 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
raw_df = ss.read.csv("smallest_crash_data.csv", header=True, inferSchema=True)

In [10]:
raw_df.printSchema()
raw_df.show(5)

root
 |-- ID: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Severity: integer (nullable = true)
 |-- Start_Time: timestamp (nullable = true)
 |-- End_Time: timestamp (nullable = true)
 |-- Start_Lat: double (nullable = true)
 |-- Start_Lng: double (nullable = true)
 |-- End_Lat: string (nullable = true)
 |-- End_Lng: string (nullable = true)
 |-- Distance(mi): double (nullable = true)
 |-- Description: string (nullable = true)
 |-- Street: string (nullable = true)
 |-- City: string (nullable = true)
 |-- County: string (nullable = true)
 |-- State: string (nullable = true)
 |-- Zipcode: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Airport_Code: string (nullable = true)
 |-- Weather_Timestamp: timestamp (nullable = true)
 |-- Temperature(F): double (nullable = true)
 |-- Wind_Chill(F): double (nullable = true)
 |-- Humidity(%): double (nullable = true)
 |-- Pressure(in): double (nullable = true)
 |-- V

## Imbalance in Data

Crash severity of 2 or 3 is much more common than any other severity ranking, and crash durations (duration of impact on traffic) are recorded as 30 minutes the majority of the time. This shows that the data is imbalanced, which we will have to consider when building our classifier.

In [7]:
severity_count = raw_df.groupBy(F.col("Severity")).count()
severity_count.show()

time_count = raw_df.withColumn("time_diff", (F.col("Start_Time").cast("long") - F.col("End_Time").cast("long"))/60).groupBy(F.col("time_diff")).count()
time_count.orderBy(F.col("count"), ascending=False).show()

+--------+-----+
|Severity|count|
+--------+-----+
|       1|    1|
|       3|   81|
|       2|  118|
+--------+-----+

+------------------+-----+
|         time_diff|count|
+------------------+-----+
|             -30.0|  151|
|             -45.0|   21|
|             -60.0|    2|
|            -235.0|    1|
|            -826.0|    1|
|-74.61666666666666|    1|
|-64.53333333333333|    1|
|            -314.0|    1|
|            -137.0|    1|
|            -198.0|    1|
|-72.98333333333333|    1|
|             -92.0|    1|
|            -350.0|    1|
|            -177.0|    1|
|            -109.0|    1|
|            -227.0|    1|
|             -72.0|    1|
|            -871.0|    1|
|            -186.0|    1|
|            -156.0|    1|
+------------------+-----+
only showing top 20 rows



## PCA

This data has a large number of features, so we will perform PCA to better understand which features of our dataset affect crash severity.

In [8]:
column_names = raw_df.columns
assembler = VectorAssembler(inputCols=column_names, outputCol="features")
assembled_data = assembler.transform(raw_df)
assembled_data.show(5)

IllegalArgumentException: Data type string of column ID is not supported.
Data type string of column Source is not supported.
Data type timestamp of column Start_Time is not supported.
Data type timestamp of column End_Time is not supported.
Data type string of column End_Lat is not supported.
Data type string of column End_Lng is not supported.
Data type string of column Description is not supported.
Data type string of column Street is not supported.
Data type string of column City is not supported.
Data type string of column County is not supported.
Data type string of column State is not supported.
Data type string of column Zipcode is not supported.
Data type string of column Country is not supported.
Data type string of column Timezone is not supported.
Data type string of column Airport_Code is not supported.
Data type timestamp of column Weather_Timestamp is not supported.
Data type string of column Wind_Direction is not supported.
Data type string of column Weather_Condition i